In [1]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm

In [2]:
test_filepath = os.path.join('C:\DATA\ComBERT\data_finetuning_spamFiltering', 'test_without_company_name_1000.csv')

### SEC-BERT without post-training vs SEC-BERT with the proposed company name masking method

In [3]:
df = pd.read_csv(test_filepath)
df

,raw_text,is_retweet,label,created_at,yyyymm,text,contains_company_name
0,$OMG 0.042 from 0.030 CHOOO CHOOO!! ??????????...,False,human,2017-09-07 15:48:59,201709,$omg 0 042 from 0 030 chooo chooo,False
1,RT @davidfaber: A split up of $UTX is a distin...,True,human,2017-09-05 20:23:26,201709,[rt] a split up of $utx is a distinct possibil...,False
2,RT @StockConsultant: $FB (FB) Facebook stock t...,True,human,2017-09-07 11:47:46,201709,[rt] $fb fb facebook stock top of range breako...,False
3,$FB ready for breakout? Check it out on Chartm...,False,human,2017-09-03 16:13:03,201709,$fb ready for breakout check it out on chartmi...,False
4,RT @OpenOutcrier: $KEM (-6.4% pre) Kemet Corpo...,True,human,2017-09-07 12:49:48,201709,[rt] $kem 6 4 pre kemet corporation lower foll...,False
...,...,...,...,...,...,...,...
995,RT @wissmarketing: When cryptoworld is going d...,True,bot,2017-09-05 08:25:09,201709,[rt] when cryptoworld is going down $vrc is st...,False
996,RT @MarginHunt: Credit Suisse started Symantec...,True,bot,2017-09-06 15:30:15,201709,[rt] credit suisse started symantec $symc at o...,False
997,RT @PeripheraI: Looks like the $KMD engines ha...,True,bot,2017-09-08 23:17:49,201709,[rt] looks like the $kmd engines have started ...,False
998,RT @trader_travis: FREE MONEY!! Another new ex...,True,bot,2017-09-06 02:10:35,201709,rt _travis: free money another new exchange is...,False


In [4]:
target_trained_with_nums = [50, 100, 200, 300, 400]
filepaths_format = os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', 'testCompanyName=n_SECBERT_{}_*', 'results_train_{}.csv')

In [5]:
target_trained_with_num = target_trained_with_nums[0]

method_name = 'CM'
filepaths = glob(filepaths_format.format(method_name, target_trained_with_num))
print(target_trained_with_num, method_name)
print('Number of filepaths: {}'.format(len(filepaths)))

dfs = []
for filepath in filepaths:
    one_df = pd.read_csv(filepath)
    one_df = one_df[(one_df['text'].str.contains('cryptocurrency'))&(one_df['true_label']==1)&(one_df['correct'] == True)]
    one_df['method_name'] = method_name
    one_df['data_size'] = target_trained_with_num
    one_df['seed_num'] = os.path.dirname(filepath)[-1]
    dfs.append(one_df)
result = pd.concat(dfs)

result

50 CM
Number of filepaths: 5


,text,true_label,predicted_label,correct,method_name,data_size,seed_num
506,rt _pivx: #pivx top 3 gains the past 24hrs 81k...,1,1,True,CM,50,1
568,rt 0r3x: does #cryptocurrency market really ne...,1,1,True,CM,50,1
591,[rt] updated: lastest #ico list #btc $btc $eth...,1,1,True,CM,50,1
597,[rt] time to trade some $pot #potcoin [url] th...,1,1,True,CM,50,1
605,[rt] new signal within 2 hour stay connected w...,1,1,True,CM,50,1
...,...,...,...,...,...,...,...
888,[rt] this is how break downs $xvg #cryptocurre...,1,1,True,CM,50,4
907,[rt] [url]#aios #cryptocurrency #ico #preico $...,1,1,True,CM,50,4
922,rt _cult: that looks like a break out to me $e...,1,1,True,CM,50,4
975,rt _margo: $okcash rocket #iot #cryptocurrency...,1,1,True,CM,50,4


In [6]:
dfs = [result]
for text in tqdm(result['text'].values):
    for method_name in ['NoPT', 'SM', 'WWM']:
        filepaths = glob(filepaths_format.format(method_name, target_trained_with_num))

        for filepath in filepaths:
            one_df = pd.read_csv(filepath)
            one_df = one_df[(one_df['text']==text)&(one_df['correct'] == False)]
            one_df['method_name'] = method_name
            one_df['data_size'] = target_trained_with_num
            one_df['seed_num'] = os.path.dirname(filepath)[-1]
            dfs.append(one_df)
result = pd.concat(dfs)
result

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:15<00:00,  4.56it/s]


,text,true_label,predicted_label,correct,method_name,data_size,seed_num
506,rt _pivx: #pivx top 3 gains the past 24hrs 81k...,1,1,True,CM,50,1
568,rt 0r3x: does #cryptocurrency market really ne...,1,1,True,CM,50,1
591,[rt] updated: lastest #ico list #btc $btc $eth...,1,1,True,CM,50,1
597,[rt] time to trade some $pot #potcoin [url] th...,1,1,True,CM,50,1
605,[rt] new signal within 2 hour stay connected w...,1,1,True,CM,50,1
...,...,...,...,...,...,...,...
975,rt _margo: $okcash rocket #iot #cryptocurrency...,1,0,False,WWM,50,3
992,[rt] should we add support for #bat $bat conte...,1,0,False,NoPT,50,0
992,[rt] should we add support for #bat $bat conte...,1,0,False,NoPT,50,4
992,[rt] should we add support for #bat $bat conte...,1,0,False,SM,50,0


In [8]:
print(len(result['text'].unique()))
print(result['text'].unique())

target_text = list(result['text'].unique())[3]
print(target_text)

24
['rt _pivx: #pivx top 3 gains the past 24hrs 81k sathoshi and climbing $210+ million marketcap now #cryptocurrency $btc $eth'
 'rt 0r3x: does #cryptocurrency market really need to swing 40 billion dollars downward did i miss some news $btc $eth $etc $mne'
 '[rt] updated: lastest #ico list #btc $btc $eth #eth #cryptocurrency #bitcoin #crypto #blockchain links here:'
 '[rt] time to trade some $pot #potcoin [url] the world s number #cannabis #cryptocurrency [url]'
 '[rt] new signal within 2 hour stay connected will take it to the moon #retweeetplease #cryptocurrency #crypto $lsk $kmd $'
 'rt : $eth following $btc both forming inverse h&amp;s #trade the break out #cryptocurrency #trade [url]'
 'rt _com: the #cryptocurrency market is back in business without the people who couldn t stomach a temp dip $btc $pivx $xem'
 '[rt] $yoyo see lowest price ever it is time to buy to get 50 in a day $btc $eth #cryptocurrency #btc #bitcoin'
 'rt _: $kmd is hot right now 80 today this crypto currency 

In [9]:
result[(result['text']==target_text)&(result['method_name']=='NoPT')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,NoPT,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,NoPT,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,NoPT,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,NoPT,50,0


In [10]:
result[(result['text']==target_text)&(result['method_name']=='SM')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,SM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,SM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,SM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,SM,50,0


In [11]:
result[(result['text']==target_text)&(result['method_name']=='WWM')]

,text,true_label,predicted_label,correct,method_name,data_size,seed_num
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,WWM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,WWM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,WWM,50,0
597,[rt] time to trade some $pot #potcoin [url] th...,1,0,False,WWM,50,0


In [12]:
for target_text in result['text'].unique():
    print(df[df['text']==target_text].iloc[0]['raw_text'])

RT @_pivx: #PIVX top 3 gains the past 24hrs @CoinMKTCap. 81k sathoshi and climbing, $210+ million marketcap now. #cryptocurrency $btc $eth…
RT @crypt0r3x: does #cryptocurrency market really need to swing 40 BILLION dollars (downward) ?  did I miss some news? $btc $eth $etc $mne…
RT @nickrruck: UPDATED: Lastest #ICO list. #BTC $BTC $ETH #ETH #cryptocurrency #bitcoin #Crypto #Blockchain Links here:… 
RT @cryptofinancing: Time to trade some $POT (#potcoin) https://t.co/3ZtHKESJyj - the world's number #cannabis #cryptocurrency https://t.co…
RT @CRYPTOEUROPE: New signal within 2 hour! Stay connected, will take it to the moon???????????? #RetweeetPlease #cryptocurrency #Crypto $lsk $kmd $…
RT @yashwanth169: $ETH following $BTC both forming inverse h&amp;s #trade the break out.#cryptocurrency #trade https://t.co/JffMgSdWP7
RT @ictjunky_com: The #cryptocurrency market is back in business, without the people who couldn't stomach a temp dip! $btc $pivx $xem… 
RT @CoinUnited: $YOYO see lowest pri